In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files('paultimothymooney/chest-xray-pneumonia/downloads/chest-xray-pneumonia.zip', path='../data/', unzip=True)

In [4]:
!cd ../data/
!ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Aug 27 16:17 sample_data
-rw-r--r-- 1 root root   65 Sep 29 19:25 kaggle.json


In [5]:
!unzip --qq '../data/chest_xray.zip' -d '../data/'


unzip:  cannot find or open ../data/chest_xray.zip, ../data/chest_xray.zip.zip or ../data/chest_xray.zip.ZIP.


In [6]:
!pip install tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 59kB/s 
     |████████████████████████████████| 4.3MB 28.3MB/s 
     |████████████████████████████████| 501kB 44.8MB/s 


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow.keras.preprocessing.image as image
import matplotlib.image as mping;
import matplotlib.pyplot as plt;
import scipy.ndimage
import seaborn as sns
import cv2
%matplotlib inline
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense,Conv2D,GaussianDropout,MaxPooling2D
from tensorflow.keras.applications import VGG16,VGG19, DenseNet121,DenseNet169,ResNet50,InceptionV3
from tensorflow.keras.callbacks import Callback, ModelCheckpoint,EarlyStopping,ReduceLROnPlateau,LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator,save_img
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from tensorflow.keras import utils
from tensorflow.keras.regularizers import l2,l1
tf.executing_eagerly()
from sklearn.metrics import classification_report, confusion_matrix

import os
import math
import csv

defaultFolder="../data/chest_xray"
#defaultFolder="../data"


In [0]:
defaultTrainPath=defaultFolder+'/train'
defaultTestPath=defaultFolder+'/test/'
defaultValPath=defaultFolder+'/val/'

In [0]:
img_width, img_height = 224, 224
IMAGE_SIZE=(224, 224,3)
batch_size=32

In [10]:
train_datagen = ImageDataGenerator(
     rescale=1. / 255,
     #rotation_range=120,
     horizontal_flip=True, 
     vertical_flip=True,
     width_shift_range=[0, 40],
     height_shift_range=[0, 40],
     zoom_range=0.2,
     brightness_range=[0.4,1.2],
     validation_split=0.2,
     fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    defaultTrainPath,
    target_size=(img_width, img_height),
    #color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [11]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    defaultTestPath,
    target_size=(img_width, img_height),
    #color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary')

Found 624 images belonging to 2 classes.


In [12]:
validation_generator = test_datagen.flow_from_directory(
    defaultValPath,
    target_size=(img_width, img_height),
    #color_mode='grayscale',
    batch_size=batch_size,
    class_mode='binary')

Found 16 images belonging to 2 classes.


In [13]:
base_model=InceptionV3(input_shape=(IMAGE_SIZE),
                                 weights='imagenet',
                                 include_top=False)
base_model.summary()

87916544/87910968 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________

In [14]:
print(len(base_model.layers))


311


In [15]:
base_model.trainable = True
# Fine tune from this layer onwards
fine_tune_at = 250

# Freeze all the layers before the `fine_tune_at` layer
#for layer in base_model.layers[:fine_tune_at]:
#    layer.trainable =  False

#for layer in base_model.layers[fine_tune_at:]:
#    layer.trainable =  True
    
bottleneck_input  = base_model.get_layer(index=0).input
bottleneck_output = base_model.get_layer(index=-1).output
bottleneck_model  = Model(inputs=bottleneck_input,outputs=bottleneck_output)
def build_model():
    model = Sequential()
    model.add(bottleneck_model)
    model.add(layers.GlobalAveragePooling2D()),
    #model.add(layers.Dropout(0.5)),
    #model.add(layers.BatchNormalization()),
    #model.add(layers.Flatten()),
    #model.add(layers.Dropout(0.25)),
    #model.add(layers.BatchNormalization()),
    #model.add(layers.Activation('relu')),
    model.add(layers.Dense(1, activation='softmax'))
    return model

#tried label smoothning but didnt help
def custom_loss(y_true, y_pred):
    return tf.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)


#model=load_model('../content/drive/My Drive/FlowerRecognition/InceptionResNetV2_model.h5')
#model.load_weights('../content/drive/My Drive/FlowerRecognition/InceptionResNetV2_weights-improvement-20-0.87.hdf5')
model = build_model()
model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=0.01),
        metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________


In [0]:
es = EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-4 less"
        min_delta=1e-4,
        # "no longer improving" being further defined as "for at least 5 epochs"
        patience=7,
        verbose=1)


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=1e-3,verbose=1,cooldown=2,mode='min')

filepath="../content/InceptionV3_weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
#model.save('../content/drive/My Drive/FlowerRecognition/effb5_model.h5')


mcp = ModelCheckpoint(
        filepath=filepath,
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        save_weights_only=True,
        mode='min',
        verbose=1)

#decay_rate =
lrlst=[]
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.25
    epochs_drop = 5.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    print("Learning Rate :",lrate)
    return lrate
  
# ...
  
lrate = LearningRateScheduler(step_decay)


In [17]:
%%time
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
print(" Train size {} , Test Size {}",STEP_SIZE_TRAIN,STEP_SIZE_VALID)
hist=model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,
                    use_multiprocessing='True',
                    workers=16,
                    callbacks=[es,mcp,lrate]
)

 Train size {} , Test Size {} 163 19
Learning Rate : 0.01
Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
162/163 [============================>.] - ETA: 1s - loss: 3.9516 - accuracy: 0.7423Epoch 1/100
Epoch 00001: val_loss improved from inf to 5.67431, saving model to ../content/InceptionV3_weights-improvement-01-0.63.hdf5
163/163 [==============================] - 224s 1s/step - loss: 3.9421 - accuracy: 0.7429 - val_loss: 5.6743 - val_accuracy: 0.6299
Learning Rate : 0.01
Epoch 2/100
162/163 [============================>.] - ETA: 1s - loss: 3.9309 - accuracy: 0.7436
Epoch 00002: val_loss improved from 5.67431 to 5.67431, saving model to ../content/InceptionV3_weights-improvement-02-0.63.hdf5
163/163 [==============================] - 209s 1s/step - loss: 3.9421 - accuracy: 0.7429 - val_loss: 5.6743 - val_accuracy: 0.6299
Learning Rate : 0.01
Epoch 3/100
162/163 [============================>.] - ETA: 1s - loss: 3.9487 - accur

In [18]:
%%time
y_test=model.predict_generator(validation_generator)

predictions=np.argmax(y_test,axis=1)
predictions
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]


CPU times: user 953 ms, sys: 241 ms, total: 1.19 s
Wall time: 1.65 s


In [19]:
predictions

['NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL',
 'NORMAL']